In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('../util')
sys.path.append('../experiments')

In [4]:
from convert_annotations import read_webanno, resolve_ellipses, webanno_to_iob_df, EMPTY_REGEX
from pathlib import Path
from datetime import datetime
import pandas as pd

In [5]:
%%time
tsv_folder = '../../data_ggponc/output_ellipses/'
#tsv_folder = '../../ggponc_v2_annotation/other_layers/ellipses_dev'

webanno_df, sentences = read_webanno(list(Path(tsv_folder).glob('*.tsv')))

CPU times: user 12.6 s, sys: 682 ms, total: 13.3 s
Wall time: 20.5 s


In [6]:
%%time

import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    iob_df = webanno_to_iob_df(webanno_df, 'detail', True, debug=False, collect_errors=False, skip_errors=True, all_columns=True)
    iob_df['id'] = iob_df.entity_id.where(~iob_df.entity_id.isna(), iob_df.spec_id)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49821/49821 [18:24<00:00, 45.11it/s]


CPU times: user 18min 34s, sys: 9.91 s, total: 18min 44s
Wall time: 18min 42s


In [10]:
# Anything containing a prefix, suffix or fragment is potentially an elliptical construct
ellipses = iob_df[~iob_df.prefix.str.match(EMPTY_REGEX) | ~iob_df.suffix.str.match(EMPTY_REGEX) | ~iob_df.fragment.str.match(EMPTY_REGEX)].copy()

In [11]:
iob_df

token_id ts_id     span  \
file                                  sentence_id                            
00_mundhoehlenkarzinom_0002.tsv       1                   1   1-1     0-19   
                                      1                   2   1-2    20-23   
                                      1                   3   1-3    24-27   
                                      1                   4   1-4    28-37   
                                      1                   5   1-5    38-43   
...                                                     ...   ...      ...   
29_adulte-weichgewebesarkome_0405.tsv 4                  26  4-26  755-756   
                                      4                  27  4-27  757-767   
                                      4                  28  4-28  768-779   
                                      4                  29  4-29  779-780   
                                      4                  30  4-30  780-781   

                                                                 token  \
file                                  sentence_id                        
00_mundhoehlenkarzinom_0002.tsv       1            Hauptrisikofaktoren   
                                      1                            für   
                                      1                            das   
                                      1                      Auftreten   
                                      1                          eines   
...                                                                ...   
29_adulte-weichgewebesarkome_0405.tsv 4                              .   
                                      4                     lipotrophe   
                                      4                    Chemikalien   
                                      4                              )   
                                      4                              .   

                                                              value  \
file                                  sentence_id                     
00_mundhoehlenkarzinom_0002.tsv       1                     Finding   
                                      1                           _   
                                      1                           _   
                                      1                     Finding   
                                      1            Specification[1]   
...                                                             ...   
29_adulte-weichgewebesarkome_0405.tsv 4                           _   
                                      4               Specification   
                                      4                   Substance   
                                      4                           _   
                                      4                           _   

                                                                   detail  \
file                                  sentence_id                           
00_mundhoehlenkarzinom_0002.tsv       1                     Other Finding   
                                      1                                 _   
                                      1                                 _   
                                      1            Diagnosis or Pathology   
                                      1                              *[1]   
...                                                                   ...   
29_adulte-weichgewebesarkome_0405.tsv 4                                 _   
                                      4                                 *   
                                      4                External Substance   
                                      4                                 _   
                                      4                                 _   

                                                  specified_by prefix suffix  \
file                                  sentence_id                  

In [12]:
%%time
results = resolve_ellipses(ellipses, iob_df)

../util/convert_annotations.py:166: PerformanceWarning: indexing past lexsort depth may impact performance.
  sentence = ellipses.loc[idx]
../util/convert_annotations.py:168: PerformanceWarning: indexing past lexsort depth may impact performance.
  full_sentence = iob_df.loc[idx]


CPU times: user 2min 11s, sys: 1.31 s, total: 2min 12s
Wall time: 2min 12s


# Manual fixes

In [461]:
#idx = results[(results.file == '01_magenkarzinom_0090.tsv') & (results.sentence_id == 22)].index
#assert len(idx) == 1
#results.at[idx[0], 'resolution'] = ['Stadium', '2', 'und', 'Stadium 3']

In [525]:
sentence_idx = pd.DataFrame(sentences, columns=['raw_sentence'], index=webanno_df.index.drop_duplicates())

In [526]:
clean_results = results.merge(right=sentence_idx, left_on=['file', 'sentence_id'], right_index=True)

In [527]:
for _, k in clean_results[clean_results.duplicated(['file', 'sentence_id'])][['file', 'sentence_id']].iterrows():
    f = k['file']
    s = k['sentence_id']
    dups = clean_results[(clean_results.file == f) & (clean_results.sentence_id == s)]
    
    for i, inner in dups.iterrows():
        for j, outer in dups.iterrows():
            if i != j and inner.span_index_start >= outer.span_index_start and inner.span_index_end <= outer.span_index_end:
                clean_results.loc[i, 'outer'] = j

In [528]:
clean_results = clean_results[clean_results.outer.isna()]

In [529]:
def resolve(row):
    out = {}
    for c in row.index:
        val = row.loc[c]
        if len(val) == 1:
            out['multi'] = False
            out[c] = val[0]
        else:
            out['multi'] = True
            if val.count(val[0]) == len(val) and c in ['full_sentence', 'raw_sentence']:
                out[c] = val[0]
            elif c in ['fragment', 'missing_prefix', 'missing_suffix']:
                out[c] = max(val)
            else:
                out[c] = val
    return pd.Series(out)

In [530]:
clean_results = clean_results.groupby(['file', 'sentence_id', 'fragment']).aggregate(tuple).applymap(list).apply(resolve, axis=1)[list(clean_results.columns.drop(['outer', 'file', 'sentence_id', 'fragment'])) + ['multi']].reset_index()

In [531]:
def get_full_res(row):
    sent = row.raw_sentence
    if row.multi:
        spans = [o for p in row.offsets for o in p]
        resolution = [r for p in row.resolution for r in p]
    else:
        spans = row.offsets
        resolution = row.resolution
    assert len(spans) == len(resolution), (spans, resolution)
    j = 0
    for span, res in zip(spans, resolution):
        start, end = span
        start += j
        end += j
        sent = sent[:start] + res + sent[end:]
        j += (start - end + len(res))
    return sent

In [532]:
clean_results['full_resolution'] = clean_results.apply(get_full_res, axis=1)

In [533]:
import numpy as np

In [534]:
ellipses_sentence_count = len(results[['file', 'sentence_id']].drop_duplicates())
sentence_count = 78090
ellipses_sentence_count, sentence_count, ellipses_sentence_count / sentence_count

(4622, 78090, 0.05918811627609169)

In [535]:
print('Ellipses', len(results))
print('Forward ellipses', len(results[results.missing_prefix & ~results.missing_suffix]), len(results[results.fragment & results.missing_prefix & ~results.missing_suffix]))
print('Backward ellipses', len(results[~results.missing_prefix & results.missing_suffix]), len(results[results.fragment & ~results.missing_prefix & results.missing_suffix]))
print('Complex ellipses', len(results[results.missing_prefix & results.missing_suffix]), len(results[results.fragment & results.missing_prefix & results.missing_suffix]))

Ellipses 5196
Forward ellipses 1933 833
Backward ellipses 2961 2751
Complex ellipses 266 185


In [536]:
import random

def show_random(df):
    i = random.randint(0, len(df) - 1)
    item = df.iloc[i]    
    assert len(item.full_span) == len(item.resolution)
    spans = []
    solutions = []
    marker = []
    if not item.multi:
        for span, res in zip(item.full_span, item.resolution):
            if span == res:
                marker.append(' ' * len(span))
            else:
                marker.append('^' * len(res))
            start_index = res.find(span)
            spans.append(span.rjust(start_index + len(span)).ljust(len(res) - start_index))
            solutions.append(res)
        print(' '.join(spans))
        print(' '.join(solutions))
        print(' '.join(marker))
        print(f'{item.file};{item.sentence_id};{" ".join(item.full_span)}', 'fragment:', item.fragment)
    print(item.raw_sentence)
    print(item.full_resolution)
    print('...........')
    print('')

In [537]:
for _ in range(0, 10):
    show_random(clean_results[clean_results.multi])

Wesentlich vor Einleitung einer palliativen Zweittherapie ist die Abwägung, ob bei Patientenwunsch, gutem Allgemeinzustand und gesicherter Versorgungssituation erneut eine antitumorale Therapie sinnvoll ist, und ob unterstützend oder alternativ eine palliativmedizinische Betreuung/Mitbetreuung unter Einbeziehen der Angehörigen und spezialisierten Palliativversorgung (ambulant SAPV Teams, stationär Palliativstationen oder Palliativmedizinische Dienste oder Hospize) einzuleiten ist (siehe hierzu auch die S3-Leitlinie zur Palliativmedizin).
Wesentlich vor Einleitung einer palliativen Zweittherapie ist die Abwägung, ob bei Patientenwunsch, gutem Allgemeinzustand und gesicherter Versorgungssituation erneut eine antitumorale Therapie sinnvoll ist, und ob unterstützend oder alternativ eine palliativmedizinische Betreuung/Mitbetreuung unter Einbeziehen der Angehörigen und unter Einbeziehen der spezialisierten unter Einbeziehen der Palliativversorgung (ambulant SAPV Teams, stationär Palliativst

In [557]:
for _ in range(0, 10):
    show_random(clean_results[clean_results.fragment])

TNM         und / oder FIGO-Stadien
TNM-Stadien und / oder FIGO-Stadien
^^^^^^^^^^^                        
05_zervixkarzinom_0044.tsv;7;TNM und / oder FIGO-Stadien fragment: True
Ansonsten kam die Leitliniengruppe überein für die Tumorstadien, wo dies möglich war, die genauen TNM und/oder FIGO-Stadien anzugeben und auf Begriffe wie fortgeschrittenes, lokal (weit) fortgeschritten und frühes Zervixkarzinom möglichst zu verzichten oder zumindest die gemeinten Stadien mitanzuführen.
Ansonsten kam die Leitliniengruppe überein für die Tumorstadien, wo dies möglich war, die genauen TNM-Stadien und/oder FIGO-Stadien anzugeben und auf Begriffe wie fortgeschrittenes, lokal (weit) fortgeschritten und frühes Zervixkarzinom möglichst zu verzichten oder zumindest die gemeinten Stadien mitanzuführen.
...........

T2            - und T3 - Kategorien
T2-Kategorien  und T3 - Kategorien
^^^^^^^^^^^^^                     
23_peniskarzinom_0056.tsv;22;T2 - und T3 - Kategorien fragment: True
Die angegebene

In [558]:
ss = clean_results[~clean_results.fragment][clean_results[~clean_results.fragment].resolution.map(lambda r: any([t for t in r if '-' in t]))]

In [559]:
for _ in range(0, 10):
    show_random(ss)

regelmäßigen Gebrauch einer Sonnencreme mit einem Lichtschutzfaktor von 17 für UV-B und     einem Breitspektrumfilter für UV-A
regelmäßigen Gebrauch einer Sonnencreme mit einem Lichtschutzfaktor von 17 für UV-B und mit einem Breitspektrumfilter für UV-A
                                                                                        ^^^^^^^^^                             
08_aktinische-keratose-und-plattenepithelkarzinom-der-haut_0125.tsv;2;regelmäßigen Gebrauch einer Sonnencreme mit einem Lichtschutzfaktor von 17 für UV-B und einem Breitspektrumfilter für UV-A fragment: False
Eine australische Studie untersuchte den regelmäßigen Gebrauch einer Sonnencreme mit einem Lichtschutzfaktor von 17 für UV-B und einem Breitspektrumfilter für UV-A [REF].
Eine australische Studie untersuchte den regelmäßigen Gebrauch einer Sonnencreme mit einem Lichtschutzfaktor von 17 für UV-B und mit einem Breitspektrumfilter für UV-A [REF].
...........

ausgewählten CLL-Patienten mit vorausgegangenen bak

In [541]:
for _ in range(0, 10):
    show_random(clean_results[~clean_results.fragment])

höhere patienten         - und läsionsbezogene Ansprechraten
höhere patientenbezogene  und läsionsbezogene Ansprechraten
       ^^^^^^^^^^^^^^^^^                                   
08_aktinische-keratose-und-plattenepithelkarzinom-der-haut_0108.tsv;22;höhere patienten - und läsionsbezogene Ansprechraten fragment: False
Hierbei sind Rotlichtlichtquellen mit einem schmalen Spektrum bei 630 nm bei einer Dosis von 37 J/cm² zu bevorzugen, da in den einschlägigen Studien durchwegs höhere patienten- und läsionsbezogene Ansprechraten (84,8-90,9% bzw. 93,6-96,3%) erreicht wurden als mit breiteren Spektren [REF] [REF] [REF] [REF].
Hierbei sind Rotlichtlichtquellen mit einem schmalen Spektrum bei 630 nm bei einer Dosis von 37 J/cm² zu bevorzugen, da in den einschlägigen Studien durchwegs höhere patientenbezogene und läsionsbezogene Ansprechraten (84,8-90,9% bzw. 93,6-96,3%) erreicht wurden als mit breiteren Spektren [REF] [REF] [REF] [REF].
...........

mikroskopisch                bzw . makrosko

In [555]:
clean_results[(clean_results.file == '00_mundhoehlenkarzinom_0091.tsv') & (clean_results.sentence_id == 24)]

,file,sentence_id,fragment,full_sentence,span_index_start,span_index_end,full_span,offsets,resolution,missing_prefix,missing_suffix,raw_sentence,multi,full_resolution
46,00_mundhoehlenkarzinom_0091.tsv,24,True,"[Mit, der, interstitiellen, Brachytherapie, ko...",24,33,"[T1, -, und, 65, -, 8%, bei, T2, -, Tumoren]","[[171, 173], [173, 174], [175, 178], [179, 181...","[T1-Tumoren, , und, 65, -, 8%, bei, T2, -, Tum...",False,True,Mit der interstitiellen Brachytherapie konnten...,False,Mit der interstitiellen Brachytherapie konnten...


In [556]:
show_random(clean_results[(clean_results.file == '00_mundhoehlenkarzinom_0091.tsv') & (clean_results.sentence_id == 24)])

T1         - und 65 - 8% bei T2 - Tumoren
T1-Tumoren  und 65 - 8% bei T2 - Tumoren
^^^^^^^^^^                              
00_mundhoehlenkarzinom_0091.tsv;24;T1 - und 65 - 8% bei T2 - Tumoren fragment: True
Mit der interstitiellen Brachytherapie konnten bei Patienten mit Frühstadien eines Zungen- oder Mundbodenkarzinoms lokale Tumorkontroll-Raten nach 5 Jahren von 79-97% bei T1- und 65-8% bei T2 - Tumoren erzielt werden [REF], [REF], [REF], [REF], [REF], [REF], [REF], [REF], [REF].
Mit der interstitiellen Brachytherapie konnten bei Patienten mit Frühstadien eines Zungen- oder Mundbodenkarzinoms lokale Tumorkontroll-Raten nach 5 Jahren von 79-97% bei T1-Tumoren und 65-8% bei T2 - Tumoren erzielt werden [REF], [REF], [REF], [REF], [REF], [REF], [REF], [REF], [REF].
...........



In [504]:
date = datetime.now().strftime("%Y%m%d_%H%M%S")

In [505]:
date

'20220803_195531'

In [ ]:
clean_results.to_excel(f'ellipses_nodup_{date}.xlsx')